In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

/home/aditya/miniconda3/envs/trtpose/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/aditya/miniconda3/envs/trtpose/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
input_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

target_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])


In [4]:
from torchvision.datasets import VOCSegmentation
from torch.utils.data import DataLoader
train_dataset = VOCSegmentation(root="/home/aditya/Deep_Learning_Projects/Revision_projects", year = "2012", image_set = 'train', download=True, transform=input_transform, target_transform=target_transform)

val_dataset = VOCSegmentation(root="/home/aditya/Deep_Learning_Projects/Revision_projects", year = "2012", image_set = 'val', download=True, transform=input_transform, target_transform=target_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)

Using downloaded and verified file: /home/aditya/Deep_Learning_Projects/Revision_projects/VOCtrainval_11-May-2012.tar
Extracting /home/aditya/Deep_Learning_Projects/Revision_projects/VOCtrainval_11-May-2012.tar to /home/aditya/Deep_Learning_Projects/Revision_projects
Using downloaded and verified file: /home/aditya/Deep_Learning_Projects/Revision_projects/VOCtrainval_11-May-2012.tar
Extracting /home/aditya/Deep_Learning_Projects/Revision_projects/VOCtrainval_11-May-2012.tar to /home/aditya/Deep_Learning_Projects/Revision_projects


In [5]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4)

In [6]:
# dataiter = iter(train_loader)
# images, labels = dataiter.next()
# print(images.shape, labels.shape)

In [7]:
def EncoderBlock(input_channel, output_channel):
    output = nn.Sequential(nn.Conv2d(in_channels=input_channel, out_channels=output_channel, padding=1, stride=1, kernel_size=3),
                             nn.BatchNorm2d(output_channel),
                             nn.LeakyReLU(),
                            nn.Conv2d(in_channels=output_channel, out_channels=output_channel, padding=1, stride=1, kernel_size=3),
                             nn.BatchNorm2d(output_channel),
                             nn.LeakyReLU()
                             )

    return output

In [8]:
def Bridge(input_channel, output_channel):
    output = nn.Sequential(nn.Conv2d(in_channels=input_channel, out_channels=output_channel, padding=1, stride=1, kernel_size=3),
                             nn.BatchNorm2d(output_channel),
                             nn.LeakyReLU(),
                            nn.Conv2d(in_channels=output_channel, out_channels=output_channel, padding=1, stride=1, kernel_size=3),
                             nn.BatchNorm2d(output_channel),
                             nn.LeakyReLU())
    return output

In [9]:
def Upsample(input_channel, output_channel):
    output = nn.Sequential(nn.ConvTranspose2d(in_channels=input_channel, out_channels=output_channel, stride=2, kernel_size=3, output_padding=1, padding=1),
                  nn.BatchNorm2d(output_channel),
                  nn.LeakyReLU())
    return output

In [10]:
def DecoderBlock(input_channels, output_channels):
    output = nn.Sequential(nn.Conv2d(in_channels=input_channels, out_channels=output_channels, stride = 1, kernel_size=3, padding=1),
                           nn.BatchNorm2d(output_channels),
                           nn.LeakyReLU(),
                           nn.Conv2d(in_channels=output_channels, out_channels=output_channels, stride = 1, kernel_size=3, padding=1),
                           nn.BatchNorm2d(output_channels),
                           nn.LeakyReLU())
    return output

In [11]:
def Out(input_channels, output_channels):
    output = nn.Sequential(nn.Conv2d(in_channels=input_channels, out_channels=output_channels, stride=1, kernel_size=3, padding=1),
                           nn.Tanh())
    return output

In [12]:
class UnetModel(nn.Module):

    def __init__(self, input_dim, output_dim, num_filters):
        super(UnetModel, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_filters = num_filters

        self.enc_1 = EncoderBlock(self.input_dim, self.num_filters*1)
        self.pool_1 = nn.MaxPool2d(kernel_size=2, padding=0, stride=2)
        self.enc_2 = EncoderBlock(self.num_filters*1, self.num_filters*2)
        self.pool_2 = nn.MaxPool2d(kernel_size=2, padding=0, stride=2)
        self.enc_3 = EncoderBlock(self.num_filters*2, self.num_filters*4)
        self.pool_3 = nn.MaxPool2d(kernel_size=2, padding=0, stride=2)
        self.enc_4 = EncoderBlock(self.num_filters*4, self.num_filters*8)
        self.pool_4 = nn.MaxPool2d(kernel_size=2, padding=0, stride=2)

        self.bridge = Bridge(self.num_filters*8, self.num_filters*16)

        self.up_1 = Upsample(self.num_filters*16, self.num_filters*8)
        self.dec_1 = DecoderBlock(self.num_filters*16, self.num_filters*8)
        self.up_2 = Upsample(self.num_filters*8, self.num_filters*4)
        self.dec_2 = DecoderBlock(self.num_filters*8, self.num_filters*4)
        self.up_3 = Upsample(self.num_filters*4, self.num_filters*2)
        self.dec_3 = DecoderBlock(self.num_filters*4, self.num_filters*2)
        self.up_4 = Upsample(self.num_filters*2, self.num_filters*1)
        self.dec_4 = DecoderBlock(self.num_filters*2, self.num_filters*1)

        self.out = Out(self.num_filters*1, output_dim)

    def forward(self, input):
        enc1 = self.enc_1(input)
        enc_1 = self.pool_1(enc1)
        enc2 = self.enc_2(enc_1)
        enc_2 = self.pool_2(enc2)
        enc3 = self.enc_3(enc_2)
        enc_3 = self.pool_3(enc3)
        enc4 = self.enc_4(enc_3)
        enc_4 = self.pool_4(enc4)
        # print(f"enc_4: {enc_4.shape}")
        bridge = self.bridge(enc_4)

        up_1 = self.up_1(bridge)
        # print(f"up_1: {up_1.shape}")
        up_1 = torch.cat([up_1, enc4], dim=1)
        # print(f"up_1: {up_1.shape}, enc_4: {enc4.shape}")
        dec_1 = self.dec_1(up_1)
        up_2 = self.up_2(dec_1)
        up_2 = torch.cat([up_2, enc3], dim=1)
        # print(f"up_2: {up_2.shape}, enc_3: {enc3.shape}")
        dec_2 = self.dec_2(up_2)
        up_3 = self.up_3(dec_2)
        up_3 = torch.cat([up_3, enc2], dim=1)
        # print(f"up_3: {up_3.shape}, enc_2: {enc2.shape}")
        dec_3 = self.dec_3(up_3)
        up_4 = self.up_4(dec_3)
        up_4 = torch.cat([up_4, enc1], dim=1)
        # print(f"up_4: {up_4.shape}, enc_1: {enc1.shape}")
        dec_4 = self.dec_4(up_4)

        output = self.out(dec_4)

        return output



In [13]:
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
unet = UnetModel(3, 20, 64).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(unet.parameters(), lr=0.0002)
writer = SummaryWriter()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma = 0.1)

In [14]:
def ValidateModel(unet, val_loader, criterion):
    validation_loss = 0.0
    for data in val_loader:
        images, masks = data
        images, masks = images.to(device), masks.to(device).squeeze(1).long()
        output = unet(images)
        loss = criterion(output, masks)
        validation_loss+=loss.item()
    unet.train()
    return validation_loss/len(val_loader)


In [15]:
def TrainModel(unet, criterion, optimizer, n_epoch=50, patience=3):
    best_loss = float("inf")
    patience_counter = 0

    for epoch in range(n_epoch):
        running_loss = 0.0
        for i, data in enumerate(train_loader):
            images, masks = data
            images, masks = images.to(device), masks.to(device).squeeze(1).long()
            # print(f"masks.shape: {masks.shape}")
            optimizer.zero_grad()
            output = unet(images)
            # print(f"output.shape: {output.shape}")
            loss = criterion(output, masks)
            loss.backward()
            optimizer.step()
            running_loss+=loss.item()
            if i % 100 == 0:
                print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}')
                writer.add_scalar('training loss', running_loss / 100, epoch * len(train_loader) + i)
                running_loss = 0

        validation_loss = ValidateModel(unet, val_loader, criterion)
        writer.add_scalar('validation', validation_loss/100, epoch*len(train_loader) + i)
        if(validation_loss>best_loss):
            best_loss = validation_loss
            patience_counter=0
            torch.save(unet.state_dict(), '/home/aditya/Deep_Learning_Projects/Revision_projects/Best_Model.pth')
        else:
            patience_counter+=1

        if (patience_counter>=patience):
            print("Early Stopping")
            break
        scheduler.step()
    print("Finished training")
    writer.close()

In [16]:
TrainModel(unet, criterion, optimizer, 5)

[1, 1] loss: 0.029


In [ ]:
torch.cuda.empty_cache()

In [ ]:
unet.load_state_dict(torch.load('/home/aditya/Deep_Learning_Projects/Revision_projects/Best_Model.pth'))
correct = 0
total = 0
with torch.no_grad():
  for data in val_loader:
    images, labels = data
    images, labels = images.to(device), labels.to(device).squeeze(1).long()
    outputs = unet(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))